In [1]:
import tensorflow as tf
import numpy as np
import math
import glob
import time
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
from dataset import *

from sklearn.datasets import load_iris
import xgboost as xgb
import sklearn as sk
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.decomposition import PCA
import mylib as ml2
from mylib import LiftNet, create_LiftNet, create_Standard_LiftNet, Standard_LiftNet, create_Standard_LiftNet_CWRU

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import pandas as pd
import plotly.figure_factory as ff

from xgboost import XGBClassifier
from sklearn.model_selection import KFold

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#------------------------default parameters-----------------------#
test_rate = 0.2
epochs = 1000

lr=0.015
momentum=0.8

decay=0.01

validation_split=0.2
steps_per_epoch=1
validation_steps=1
bunch_steps = 100
snapshot = 500
bunch_steps = 100
snapshot = 500
print('set default parameters')

set default parameters


In [161]:
#------------------------adjustable parameters-----------------------#
model_path = ['/home/silver-bullet/newpaper/model/circle_1_to_6_in_our_data/', 
              '/home/silver-bullet/newpaper/model/circle_7_to_16_in_our_data_with_0.01_noise_data/',
              '/home/silver-bullet/newpaper/model/model_in_CWRU_data/']
model_head_name = ['Standard_LiftingNet_',
                   'Standard_LiftingNet_use_expansion_data__with_',
                   'Standard_expansion_CWRU_data_LiftingNet__with_',
                  'Standard_LiftingNet_use_expansion_data_',
                  'Standard_expansion_CWRU_data_LiftingNet_']

data_path = ['/home/silver-bullet/newpaper/data/dataset/','/home/silver-bullet/newpaper/data/CWRUdataset']

class_num = 6
circle_num = 16
cutsize = 256
steps = 600
whether_use_CWRU_data = 1
LiftingNet_noise_scale = 2
whether_expansion_data = 1
expansion_data_number = 500
noise_scales = 0.01
if whether_use_CWRU_data==1:
    noise_scales = LiftingNet_noise_scale

#artificial_feature_method: 1 is 19 features, 2 is 9 features
artificial_feature_method = 2

pca_parameters = 27

if whether_use_CWRU_data == 1:
    cutsize = 1024
    input_shape = (cutsize,2)
    channel = 2
    dataset, label = load_CWRU_data(data_path[1])
    read_model_name = model_path[2]+model_head_name[2]+str(LiftingNet_noise_scale)+'_noise_'+str(cutsize)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
    if LiftingNet_noise_scale == 0:
        read_model_name = model_path[2]+model_head_name[4]+str(cutsize)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
else:
    cutsize = 256
    input_shape = (640*circle_num,3)
    channel = 3
    dataset, label = load_dataset(data_path[0], circle_num=circle_num, cutsize=cutsize)
    if circle_num<7:
        read_model_name = model_path[0] + model_head_name[0]+str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
    else:
        read_model_name = model_path[1] + model_head_name[1]+str(LiftingNet_noise_scale)+'_noise_'+str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'

if circle_num==8:
    read_model_name= model_path[1] + model_head_name[3] +  str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'
if circle_num==12:
    read_model_name= model_path[1] + model_head_name[0] +  str(circle_num)+'_data_the_'+str(steps)+'th_snapshot_with_'+str(steps_per_epoch)+'_steps_per_epoch.h5'

print('set adjustable parameters')

dataset shape :  (832, 1024, 2)
label shape :  (832, 1)
set adjustable parameters


In [162]:
#------------------------expansion data parameters-----------------------#

if whether_expansion_data == 1:
    dataset, label = expansion_and_add_noise(dataset,label,exnumber=expansion_data_number, noise_scales=noise_scales)
    print('data expansion')

#------------------------data processing---------------------------------#

input_shape = (dataset.shape[1],dataset.shape[2])
channel = dataset.shape[2]

x_number = dataset.shape[0]

x = dataset
print('x.shape: ',x.shape)
y = label
print('y.shape: ', y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_rate, random_state=615)
print('x_train.shape: ', x_train.shape)
print('x_test.shape: ',x_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

x_train_number = x_train.shape[0]
x_test_number = x_test.shape[0]
print('x_train_number: ',x_train_number)
print('x_test_number: ',x_test_number)

(237, 1024, 2)
(118, 1024, 2)
(117, 1024, 2)
(118, 1024, 2)
(118, 1024, 2)
(118, 1024, 2)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(500, 1024, 2)
(500, 1)
(3000, 1024, 2)
(3000, 1)
data_max_abs_x.shape:  (3000, 1, 2)
noise_data.shape:  (3000, 1024, 2)
after expansion: 
dataset shape :  (3832, 1024, 2)
label shape :  (3832, 1)
data expansion
x.shape:  (3832, 1024, 2)
y.shape:  (3832, 1)
x_train.shape:  (3065, 1024, 2)
x_test.shape:  (767, 1024, 2)
y_train.shape:  (3065, 1)
y_test.shape:  (767, 1)
x_train_number:  3065
x_test_number:  767


In [163]:
#--------------------------------load LiftingNet-------------------------------------------#
if whether_use_CWRU_data == 1:
    liftnet = create_Standard_LiftNet_CWRU(class_num = class_num, 
                                       channel = channel, 
                                       cut_size = cutsize, 
                                       input_shape = input_shape,
                                       lr=lr, 
                                       momentum = momentum, 
                                       decay=decay)
else:
    liftnet = create_Standard_LiftNet(class_num = class_num, 
                                  channel = channel, 
                                  circle_num = circle_num, 
                                  input_shape=input_shape,
                                  lr=lr, 
                                  momentum=momentum, 
                                  decay=decay)

liftnet.load_weights(read_model_name)
print('load model')

Model: "LiftNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1024, 2)]         0         
_________________________________________________________________
convinputs (Conv1D)          multiple                  6         
_________________________________________________________________
predict1_1 (Conv1D)          multiple                  14        
_________________________________________________________________
predict1_2 (Conv1D)          multiple                  30        
_________________________________________________________________
update1_1 (Conv1D)           multiple                  62        
_________________________________________________________________
update1_2 (Conv1D)           multiple                  30        
_________________________________________________________________
predict2_1 (Conv1D)          multiple                  310 

In [164]:
print('------------------------------------------------------------')
print('first predict: ')

#test
t1 = time.time()
pre_y = liftnet.predict(x_test, steps=1)
t2 = time.time()
#print(pre_y)
print('LiftingNet_noise_scale: ', LiftingNet_noise_scale)
ml2.evaluate_model(pre_y,y_test)

pre_y2 = ml2.pre_to_index(pre_y)

# 计算准确率

cnt1 = 0
cnt2 = 0

for i in range(len(y_test)):
    if pre_y2[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))
print('test time: '+str(t2-t1)+'s')
del liftnet


print('save and delete model')
print('------------------------------------------------------------')
print('program end')

------------------------------------------------------------
first predict: 
LiftingNet_noise_scale:  2
normal result:
tp: 140  tn: 9  fp: 63  fn: 555
tp/(tp+tn):  0.9395973154362416
tp/(tp+fp):  0.6896551724137931
inner_ring result:
tp: 85  tn: 30  fp: 16  fn: 636
tp/(tp+tn):  0.7391304347826086
tp/(tp+fp):  0.8415841584158416
outer_ring result:
tp: 89  tn: 40  fp: 31  fn: 607
tp/(tp+tn):  0.689922480620155
tp/(tp+fp):  0.7416666666666667
roller result:
tp: 63  tn: 50  fp: 34  fn: 620
tp/(tp+tn):  0.5575221238938053
tp/(tp+fp):  0.6494845360824743
joint result:
tp: 104  tn: 42  fp: 52  fn: 569
tp/(tp+tn):  0.7123287671232876
tp/(tp+fp):  0.6666666666666666
Accuracy: 71.97 % 
test time: 3.093766212463379s
save and delete model
------------------------------------------------------------
program end
